In [34]:
import numpy as np
np.seterr(divide='ignore');

In [35]:
a = 2
U0 = 1.5
E0 = -0.95
precision = 10e-9

In [36]:
def f(E):
	return (1 / np.tan(np.sqrt(2 * a ** 2 * U0 * (1 + E / U0)))) - np.sqrt(-U0 / E - 1)

def df(E):
	return - (a ** 2) / (
			(np.sin(np.sqrt(2 * a * 2 * U0 * (1 + E / U0))) ** 2) * np.sqrt(2 * a ** 2 * U0 * (1 + E / U0))) - (
				   U0 / 2) / (E ** 2 * ((-U0 / E) - 1))

Начальный интервал для метода дихотомии находим из $\cot(\sqrt{2a^2U_{0}(1 + \frac{E}{U_{0}})}) = 0$
Тогда $\sqrt{2a^2U_{0}(1 + \frac{E}{U_{0}})} = \frac{\pi}{2} + \pi n$
И $E = \frac{\pi^2}{8a^2} - U_{0}$

In [37]:
def dichotomy(a, b, error=precision):
	middle = (a + b) / 2
	curError = (b - a) / 2
	if curError < error:
		return middle
	return dichotomy(a, middle) if f(middle) * f(a) <= 0 else dichotomy(middle, b)

x_0 = -U0
x_1 = (np.pi ** 2) / (8 * a ** 2) - U0
dichotomyRoot = dichotomy(x_0, x_1)

# в качестве начального приближения возьмем корень, полученный методом дихотомии
x_initial = dichotomyRoot
Lambda = 1 / df(x_initial) + 0.001

def fixedPointIteration(x, x0=x_initial, error=precision):
	if df(x) < 1:
		value = x - Lambda * f(x)
		curError = np.abs(x0 - x)
		return fixedPointIteration(value) if curError > error else x

fixPointIterRoot = fixedPointIteration(E0)

def Newton(x, error=precision):
	value = x - f(x) / df(x)
	curError = np.abs(fixPointIterRoot - x)
	return Newton(value) if curError > error else x

newtonRoot = Newton(E0)

In [38]:
print('NewtonMethod              ' + str(newtonRoot))
print('f({}) = {}'.format(newtonRoot, f(newtonRoot)))
print('FixedPointIterationsMethod' + str(fixPointIterRoot))
print('f({}) = {}'.format(fixPointIterRoot, f(fixPointIterRoot)))
print('DichotomyMethod           ' + str(dichotomyRoot))
print('f({}) = {}'.format(dichotomyRoot, f(dichotomyRoot)))

NewtonMethod              -1.3160669823541098
f(-1.3160669823541098) = -2.8810002661305845e-08
FixedPointIterationsMethod-1.3160669879566542
f(-1.3160669879566542) = -1.2642940028229077e-09
DichotomyMethod           -1.316066997326563
f(-1.316066997326563) = 4.4804200849402065e-08
